# Gipsz Jakab (ABC123)

# A labor célja

A labor célja egy rövid bevezetőt adni a manapság oly népszerű "data science" Python eszközeibe.
A labor feladatai előtt mindenképp meg kell ismerkedni a Python nyelv alapjaival. A laborhoz tartozó rövid magyar Python bevezetőt [itt](https://github.com/juditacs/labor/blob/master/uzleti_intelligencia/Python_alapok.ipynb) találod.

A pandashoz egy rövid magyar bevezető [itt](https://github.com/juditacs/labor/blob/master/uzleti_intelligencia/Pandas_alapok.ipynb).

A labort összeállította: Ács Judit

# A labor menete

A labor elején együtt fogunk dolgozni az első feladatokon. A labor második részében önállóan kell megoldanotok a notebookban található kérdéseket. 5 feladat van, amelyek 5 kérdést tartalmaznak. A kérdések sorszámozva vannak és Q-val kezdődnek: Q1.1-Q5.5-ig. Néhány szorgalmi kérdés is van, ezekkel kiválthatók a hiányzó kérdések a kötelezők közül.

Egy feladat sikeres teljesítéséhez legalább 3 kérdésre kell helyesen válaszolni.

A laborhoz külön jegyzőkönyvet nem kell készíteni, ezt a notebookot kell vasárnap estig az AUT portálra feltölteni. A fájlrendszerben `uzleti_intelligencia.ipynb` néven megtaláljátok a notebookot abban a könyvtárban, ahonnan indítottátok a jupytert.

In [ ]:
import pandas as pd  # konvenció szerint pd aliast használunk
%matplotlib inline
import matplotlib
import numpy as np

# tegyük szebbé a grafikonokat
matplotlib.style.use('ggplot')
#pd.set_option('display.mpl_style', 'default')
matplotlib.pyplot.rcParams['figure.figsize'] = (15, 3)
matplotlib.pyplot.rcParams['font.family'] = 'sans-serif'

A MovieLens adatsorral fogunk dolgozni, de először le kell töltenünk. http://grouplens.org/datasets/movielens/

Csak akkor töltjük le a fájlt, ha még nem létezik.

In [ ]:
import os

if not os.path.exists('ml.zip'):
    import urllib
    u = urllib.request.URLopener()
    u.retrieve("http://files.grouplens.org/datasets/movielens/ml-100k.zip", "ml.zip")

Kicsomagoljuk:

In [ ]:
if not os.path.exists('ml-100k'):
    from zipfile import ZipFile
    with ZipFile('ml.zip') as myzip:
        myzip.extractall()

## Adat betöltése és normalizálása

A pd.read_table függvény táblázatos adatok betöltésére alkalmas. Több tucat paraméterrel [rendelkezik](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_table.html), de csak egy kötelező paramétere van: a fájl, amit beolvasunk.
A karakterkódolást is meg kell adnunk, mert a fájl nem az alapértelmezett (utf-8) kódolást használja, hanem az ISO-8859-1-et, vagy köznéven a latin1-et.

In [ ]:
# df = pd.read_table("ml-100k/u.item")  # UnicodeDecodeErrort kapunk, mert rossz dekódert használ
df = pd.read_table("ml-100k/u.item", encoding="latin1")
df.head()

Ez még elég rosszul néz ki. Hogyan tudnánk javítani?
1. Rossz szeparátort használt a függvény (tab az alapértelmezett). A fájlban | a szeparátor. Ezt a `sep` paraméterrel tudjuk megadni.
1. A fájl első sora került az oszlopnevek helyére. Az oszlopok valódi nevei a README fájlból derülnek ki, amit kézzel megadhatjuk a read_table-nek a `names` paraméterben.
1. A read_table automatikusan generált egy id-t minden sornak, azonban az adatfájlban a filmek már rendelkeznek egy egyedi azonosítóval (movie_id), használjuk ezt a DataFrame indexeként (`index_col` paraméter). Célszerű szóköz nélküli, kisbetűs oszlopneveket használni, mert akkor attribútumként is elérjük őket (df.release_date).

In [ ]:
column_names = [
    "movie_id", "title", "release_date", "video_release_date", "imdb_url", "unknown", "action", "adventure", "animation",
    "children", "comedy", "crime", "documentary", "drama", "fantasy", "film_noir", "horror", "musical", "mystery",
    "romance", "sci_fi", "thriller", "war", "western"]

In [ ]:
df = pd.read_table("ml-100k/u.item", sep="|", names=column_names, encoding="latin1", index_col='movie_id')
df.head()

Két oszlop is van, amik dátumot jelölnek: `release_date, video_release_date`. A pandas parszolni tudja a dátumokat többféle népszerű formátumban, ehhez csak a `parse_dates` paraméterben kell megadnunk a dátumot tartalmazó oszlopokat. Figyeljük meg, hogy ahol nincs dátum, az Nan (not a number)-ről NaT-ra változik.

In [ ]:
df = pd.read_table("ml-100k/u.item", sep="|", names=column_names, encoding="latin1", parse_dates=[2,3], index_col='movie_id')
df.head()

Még mindig nem tökéletes, hiszen a filmek címei után szerepel az évszám zárójelben, ami egyrészt redundáns, másrészt zaj. Tüntessük el!

A szokásos `str` műveletek egy része elérhető DataSeries objektumokra is (minden elemre végrehajtja). A függvényeket az `str` névtérben találjuk.

In [ ]:
df.title.str

Egy reguláris kifejezéssel eltüntetjük a két zárójel közti részt, majd eltávolítjuk az ott maradt whitespace-eket (a strip függvény a stringek elejéről és végéről is eltávolítja).
Végül adjuk értékül a régi title oszlopnak a kezdő és záró whitespace-ektől megfosztott változatát.

In [ ]:
df.title = df.title.str.replace(r'\(.*\)', '').str.strip()

In [ ]:
df.head()

A `video_release_date` mező az első néhány sorban csak érvénytelen mezőket tartalmaz. Vajon igaz ez az egész DataFrame-re? Listázzuk ki azokat a mezőket, ahol nem NaT a video_release_date értéke, vagyis érvénytelen dátum.

In [ ]:
df[df.video_release_date.notnull()]

Nincs ilyen mező, ezért elhagyhatjuk az oszlopot.

In [ ]:
df = df.drop('video_release_date', axis=1)
df.head()

Van egy `unknown` oszlop, ettől is szabaduljunk meg!

In [ ]:
df = df.drop('unknown', axis=1)

## Adatok felszínes vizsgálata

Nézzük meg, hogy milyen információkat tudhatunk könnyedén meg a DataFrame-ről.

A `describe` függvény oszloponként szolgáltat alapvető infomációkkal: darabszám, átlag, szórás stb.
Mivel a legtöbb mező bináris, most nem tudunk meg sok hasznos információt a mezőkről.

In [ ]:
df.describe()

### Átlag, szórás, variancia stb.

Egyenként is lekérdezhetőek:

1. count()
1. átlag: mean()
1. szórás: std()
1. variancia: var()
1. 50% kvantilis: quantile(.5)
1. min-max

In [ ]:
df.quantile(.9).head()

# 1. feladat: Egyszerű lekérdezések 

## Melyik filmek jelentek meg 1956-ban? 

In [ ]:
df[df.release_date.dt.year == 1956]

## Több feltétel

A szűrési feltételeket tudjuk kombinálni az & és a | operátorokkal. Az operátorok a Boole algebra szerint működnek igazságértékekre:

In [ ]:
print(True & False)
print(True | False | False)
print(True | False & False)

Az & precedenciája magasabb, mint a |-é. Mindkét operátor precedenciája magasabb, mint az összehasonlító operátoroké (==, <, >), ezért mindig zárójelezzünk!

## Melyik filmek jelentek meg a 80-as években?

In [ ]:
d = df[(df.release_date.dt.year >=  1980) & (df.release_date.dt.year < 1990)]
d.shape

107 film jelent meg a 80-as években, ezt már nem praktikus kiíni. Nézzük meg csak az első 3-at.

In [ ]:
d.head(3)

### A megjelenítési év legyen külön oszlop

Többször fogjuk még használni a megjelenési évet, ezért praktikus külön év oszlopot létrehozni.

A DateTime mezőhöz használható metódusok és attribútumok a dt névtérben vannak, így tudjuk minden oszlopra egyszerre meghívni. Az eredményt egy új oszlopban tároljuk.

In [ ]:
df['year'] = df.release_date.dt.year

## Mikor jelentek meg a Die Hard filmek? 

In [ ]:
df[df.title == 'Die Hard']

Sajnos csak teljes egyezésre tudunk így szűrni.

A szöveges mezőkre a pandas nyújt egy csomó műveletet, amik az str névtérben vannak (ahogy a dátum mezőkre a dt-ben voltak).

In [ ]:
df[df.title.str.contains('Die Hard')]

A Die Hard 4 és 5 hiányzik. Kilógnának az adatsorból? Nézzük meg még egyszer, hogy mikori filmek szerepelnek.

In [ ]:
df.release_date.describe()

A Die Hard 4 és 5 2007-ben, illetve 2013-ban jelentek meg, ezért nem szerepelnek az adatban.

### Melyik filmek tartoznak egyszerre az akció és romantikus kategóriába?

In [ ]:
d = df[(df.action==1) & (df.romance==1)]
print(len(d))
d.head()

### Melyik filmek tartoznak az akció VAGY a romantikus kategóriába?

Itt a Boole vagyra gondolunk.

In [ ]:
d = df[(df.action==1) | (df.romance==1)]
print(len(d))
d.head()

## Q1.1. Hány akciófilm jelent meg 1985 előtt, illetve 1985-ben vagy később?

## Q1.2. Létezik-e gyerekeknek szóló thriller? Ha igen, melyek azok?

## Q1.3. Hány filmnek hosszabb a címe, mint 30 karakter?

## Rendezés

A DataFrame-eket rendezésére gazdag eszköztárat kínál a pandas. A legegyszerűbb rendezés az index szerinti:

In [ ]:
df.sort_index().head()

Bármelyik oszlop szerint is rendezhetünk:

In [ ]:
print(df.sort_values('title').title.head())
print(df.sort_values('title').title.tail())

## Q1.4. Melyik a legrégebbi és a legújabb film?

A megjelenésnek nem csak éve van!

## Q1.5. Melyik a legrégebbi és a legújabb horrorfilm?

Elég egy megfelelő DataFrame head/tail függvényével listázni.

# 2. feladat: Csoportosítás és vizualizáció

## Hány filmet adtak ki évente?

A kérdést két lépésben tudjuk megválaszolni:

1. csoportosítás évenként
2. összesítés 1-1 évre


In [ ]:
df.groupby('year').count().plot(y='title')

Vonaldiagram az alapértelmezett, de oszlopdiagramként informatívabb lenne.

In [ ]:
df.groupby('year').count().plot(y='title', kind='bar')

Lásztik, hogy a 80-as évek végén nőtt meg a kiadott filmek száma, kicsit közelítsünk rá. Ehhez először szűrni fogjuk a 1985 utáni filmeket, majd csoportosítva ábrázolni.

In [ ]:
d = df[df.year > 1985]
d.groupby('year').count().plot(y='title', kind='bar')
# df[df.year > 1985].groupby('year').count().plot(y='title', kind='bar')  # vagy egy sorban

## Groupby tetszőleges feltétel szerint

Nem csak egy kategóriaértékű oszlop szerint csoportosíthatunk, hanem tetszőleges kifejezés szerint. Ezt kihasználva fogunk évtizedenként csoportosítani. A groupby-nak bármilyen kifejezést megadhatunk, ami diszkrét értékekre képezi le a sorokat, tehát véges sok csoport egyikébe helyezi (mint egy hash függvény).

Az évtizedet úgy kaphatjuk meg, ha az évet 10-zel osztjuk és csak az egészrészt tartjuk meg, hiszen 1983/10 és 1984/10 egészrésze ugyanúgy 198. Használjuk a Python egészosztás operátorát (//).

In [ ]:
d = df.groupby(df.year // 10 * 10)
d.groups.keys()  # létrejött csoportok listázása

In [ ]:
d.count().plot(y='title', kind='bar')

## Q2.1. Hány vígjátékot (comedy) adtak ki évente?

## Gyerekfilmet vagy krimit adnak ki többet évtizedenként?

In [ ]:
col1 = 'children'
col2 = 'crime'
d = df[['year', col1, col2]].copy()
d['diff'] = d[col1] - d[col2]
d.groupby(d.year // 10 * 10).sum()  #.plot(y='diff', kind='bar')

A 90-es években több filmet adtak ki, mint előtte összesen, nézzük meg azt az évtizedet közelebbről!

## Q2.2. Mennyivel adtak ki több gyerekfilmet, mint krimit évente a 90-es években? Ábrázold.

## Q2.3. Melyik műfajból hányat adtak ki? 

Az évnek nyilván nincs értelme.
Messze drámából volt a legtöbb.

## Q2.4. Hagyományos lexikont szeretnénk készíteni a filmekből. Melyik kezdőbetű hányszor fordul elő a filmek címében? Ábrázold tortadiagramon.


A tortadiagramhoz érdemes megváltoztatni a diagram képarányát, amit a plot függvény `figsize` paraméterének megadásával tehetsz meg. `figsize=(10,10)`. Százalékokat a `autopct="%.0lf%%"` opcióval lehet a diagramra írni.

A tortadiagramot szebbé teheted másik colormap választásával: [dokumentáció](http://pandas.pydata.org/pandas-docs/stable/visualization.html#colormaps) és a [colormapek listája](http://scipy.github.io/old-wiki/pages/Cookbook/Matplotlib/Show_colormaps).

## Q2.5. Ábrázold a kiadási hónapok eloszlását egy tortadiagramon!


# 3. feladat: pd.merge

Az adathalmaz lényegi része a 100000 értékelés, amit az u.data fájlból tudunk beolvasni. A README-ből kiolvashatjuk a fájl oszlopait.

In [ ]:
cols = ['user', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-100k/u.data', names=cols)

In [ ]:
ratings.head()

A timestamp oszlop Unix timestampeket tartalmaz, konvertáljuk DateTime-má.

In [ ]:
ratings['timestamp'] = pd.to_datetime(ratings.timestamp, unit='s')
ratings.head()

## Merge a film táblával

Mivel már több DataFrame-mel dolgozunk, érdemes a filmeket tartalmazó táblának beszédesebb nevet adni.

In [ ]:
movies = df

A pd.merge függvény képes mindenféle join műveletre. Ennek is rengeteg paramétere van, de most csak néhány alapvető művelettel fogunk megismerkedni.

A merge-nek van egy bal és egy jobb DataFrame-je, ami most legyen a ratings, illetve a movies. A ratings DataFrame-ben a movie_id oszlophoz kell megkeresni a movies DataFrame megfelelő indexű sorát. A merge megkülönbözteti azt, hogy index vagy nem index oszlop mentén egyesít:
1. a left_on/right_on paraméterrel adhatjuk meg azt a bal/jobboldali nem index oszlopot, ami mentén merge-ölünk,
2. a left_index/right_index=True flaggel jelezhetjük, ha index mentén akarunk merge-ölni.

In [ ]:
r = pd.merge(ratings, movies, left_on='movie_id', right_index=True)

Meghívhatjuk a baloldali DataFrame metódusaként is:

In [ ]:
r = ratings.merge(movies, left_on=['movie_id'], right_index=True)

Csak néhány oszlop érdekel minket a movies DataFrame-ből:

In [ ]:
r = ratings.merge(movies[['title', 'year', 'release_date']], left_on=['movie_id'], right_index=True)

## Hány értékelés érkezett a film megjelenése előtt?

In [ ]:
len(r[r.timestamp <= r.release_date])

## Hogy oszlik meg ez a szám a filmek között? 

In [ ]:
r[r.timestamp <= r.release_date].title.value_counts()

In [ ]:
movies[movies.title == 'Apt Pupil']

## Q3.1. Hány film kapott legalább egyszer 4 fölötti értékelést?

## Hisztogram készítése az egyes értékelésekről

Hisztogram készítésére (melyik érték hányszor szerepelt), a hist függvény áll rendelkezésünkre:

In [ ]:
ratings.hist('rating')

## Q3.2. Ábrázold hisztogramon a thrillerek értékeléseit!

Vigyázat: az előzőleg merge-ölt táblázat nem tartalmaz műfaji információkat.

## Q3.3. Mi az értékelések átlaga évtizedenként (film megjelenési éve)? 

## Q3.4. Az értékelésekhez tartozik egy timestamp. Mi az értékelések átlaga hónapokra bontva?

Tehát melyik hónapban jószívűbbek az emberek?

## Q3.5. Melyik hónapban legnagyobb a gyerekfilmek értékeléseinek szórása? 

# 4. feladat: Users DataFrame

## Q4.1 Olvasd be a u.user fájlt egy users DataFrame-be!

Segítségképpen az oszlopok: user_id, age, gender, occupation, zip

## Q4.2. Hogy oszlanak el az értékelések a különböző foglalkozású emberek között? Ábrázold tortadiagramon.

Pl. az értékelések 13%-át az írók adják (nem ez a valódi érték).

## Q4.3. Korcsoportonként hány értékelést adtak le? 10 évet veszünk egy korcsoportnak, tehát 10-19, 20-29 stb. Ábrázold oszlopdiagramon.

## Q4.4. Milyen átlagos értékelést adnak az egyes szakmák képviselői?

## Q4.5. Készíts hisztogramot az értékelési kedvről! Hány user adott le N értékelést?

## Q4.6. (Szorgalmi) Milyen volt a nemek eloszlása a romantikus filmet, illetve az akciófilmeket értékelők között? Készíts két tortadiagramot!

## Q4.7. (Szorgalmi) Jóval több férfi adott le értékelést. Hogy alakulnak ezek az arányok, ha normálunk az összes értékelésre jellemző nemek arányával?

# 5. feladat: K legközelebbi szomszéd

Ebben a feladatban a műfajok alapján fogjuk megkeresni minden filmhez a hozzá leghasonlóbb K filmet. Az eljárás neve k-nearest neighbor (KNN). A scikit-learn tartalmaz több KNN implementációt is, mi most a ball_tree-t fogjuk használni. Az osztály dokumentációja itt található: http://scikit-learn.org/stable/modules/neighbors.html

## Q5.1. Nyerd ki a movies adattáblából a műfaji címkéket mátrixként!

A DataFrame as_matrix függvényével készíthetünk mátrixot a DataFrame-ből. Az as_matrixnak megadható, hogy melyik oszlopok kerüljenek bele a mátrixba. Most csak a műfajokat tartalmazó oszlopokat kell megtartani. Vigyázz, az utolsó oszlop az évet tartalmazza!

A mátrix neve legyen X.

## Q5.2. Futtasd le a KNN-t az X mátrixon!

Ehhez bele kell nézned a NearestNeighbors [dokumentációjába](http://scikit-learn.org/stable/modules/neighbors.html).

Az indexeket az indices változóban tárold.

A legközelebbi szomszédok számát a K változóban tárold. Először állítsd 4-re a K-t, később kísérletezhetsz más értékekkel is.

Az indices változó tartalmazza az indexeket, ebből készítsünk DataFrame-et:

In [ ]:
ind = pd.DataFrame(indices)
ind.head()

Értelmezzük a táblázatot!
Az index oszlop (első oszlop) azt mondja meg, hogy az X mátrix hányadik sorához tartozó szomszédok találhatók meg a sorban. A 0-3. nevű oszlopok a legközelebbi szomszédokat adják meg. Legtöbb film esetén saját maga a legközelebbi szomszédja, hiszen 0 a távolságuk, azonban nincs mindenhol így. Mit gondolsz, miért?

A táblázat indexe 0-val kezdődik, de a movies táblában a movie_id 1-től indul.

## Q5.3. Állítsd át az ind DataFrame indexét úgy, hogy 1-től indexeljen! Az összes mezőt is növeld meg eggyel!

Segítség: a Pandas alapok [Vektoros műveletvégzés](http://github.com/juditacs/labor/tree/master/uzleti_intelligencia/Pandas_alapok.ipynb#Vektoros-műveletvégzés) részét érdemes megnézni.

## Q5.4. Keresd meg az indexekhez tartozó filmcímeket!

Itt több merge-re lesz szükségünk, K db szomszéd címeit kell megkeresnünk. Hány merge hívás kell?

Segítség: a merge-ölések sorozatát meg lehet oldani egy 4 soros rekurzív függvénnyel.

## Q5.5. Jelenjen meg a táblázatban az a film is, aminek a szomszédjai a sorban vannak! A címeken kívül más oszlopa ne legyen a táblázatnak!

Most olyanok a soraink, hogy: movie_id | nearest1 | nearest 2 stb.

A movie_id mellett jó lenne az eredeti címet is látni.

## Q5.6. (Szorgalmi) Mik a Die Hardhoz leghasonlóbb filmek?

# Beadás

Amennyiben nem sikerült a végére érned, de szeretnéd befejezni, vasárnapig megteheted a saját gépeden. Telepítési instrukciókat a Python alapok notebookban találsz.

Mielőtt feltöltöd a fájlt:
1. próbáld ki, hogy újraindított kernellel is működik-e. Ezt a Kernel->Restart & Run All opcióval teheted meg.
1. Ügyelj, hogy ne maradjanak hosszú táblázatok kiírva. Helyette használd a head függvényt.

A laborhoz jegyzőkönyvként ez a notebook fájl szolgál. Az `uzleti_intelligencia.ipynb` fájlt csomagold be és töltsd fel az AUT portálra vasárnapig.

# Visszajelzés

Köszönöm, hogy végigcsináltad a labort, remélem, hogy hasznosnak találtad.

Minden visszajelzés fontos és be fogom építeni következő alkalmakba. Kérlek töltsd ki ezt a rövid [kérdőívet](https://docs.google.com/forms/d/1QKKR_qsP0850H0lY_aGPHXnN2YnJgKMvkH37uNGtQ9s/viewform).

# Folytatás

Ha kedvet kaptál a témához, érdemes megnézni a [kaggle.com](https://www.kaggle.com/) data science oldalt, ahol megnézheted mások kódját, részt vehetsz versenyeken és sokat tanulhatsz a témáról.